In [23]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Flatten,Dense,Dropout,Rescaling
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.applications.densenet import DenseNet121
from tensorflow.keras.utils import image_dataset_from_directory
import pathlib
import cv2
import numpy as np

In [24]:
data_path=pathlib.Path('datasets/stanford_dogs/images/images/Images')

In [25]:
train_ds=image_dataset_from_directory(data_path,validation_split=0.2,subset='training',seed=123,image_size=(224,224),batch_size=16)
test_ds=image_dataset_from_directory(data_path,validation_split=0.2,subset='validation',seed=123,image_size=(224,224),batch_size=16)

Found 20580 files belonging to 120 classes.
Using 16464 files for training.
Found 20580 files belonging to 120 classes.
Using 4116 files for validation.


In [26]:
train_ds

<BatchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None,), dtype=tf.int32, name=None))>

In [27]:
for x,y in train_ds.take(1) :
    images_np = x.numpy()
    labels_np = y.numpy()
    print("x :",images_np)
    print("x type:",images_np.dtype)
    print("y :",labels_np)

x : [[[[ 76.64321    68.64321    81.64321  ]
   [ 74.48643    66.48643    79.48643  ]
   [ 81.23914    74.60968    86.92441  ]
   ...
   [ 62.904316   57.701008   86.497696 ]
   [129.49774   108.62026   127.67596  ]
   [132.37923   103.72738   114.73849  ]]

  [[ 87.68321    79.68321    92.68321  ]
   [ 89.20551    81.20551    94.20551  ]
   [ 80.188385   73.558914   85.87365  ]
   ...
   [ 63.737133   58.533825   87.33052  ]
   [131.84938   112.649445  131.70514  ]
   [134.7445    108.08155   118.429695 ]]

  [[111.72649   100.72649   114.72649  ]
   [ 97.78439    89.54332   102.62368  ]
   [ 85.842995   77.76712    90.79241  ]
   ...
   [ 59.14937    55.94606    85.42799  ]
   [140.12868   121.251205  140.3069   ]
   [134.6838    108.64164   119.17669  ]]

  ...

  [[ 86.785965  102.785965  137.97824  ]
   [ 80.48876    96.48876   132.30663  ]
   [ 86.38543   102.38543   137.62054  ]
   ...
   [ 73.6242     79.15539   103.38979  ]
   [ 84.11435    88.275055  113.1947   ]
   [ 92.2388

In [28]:
cv2.imshow('src',images_np[0].astype(np.uint8))
cv2.waitKey()
cv2.destroyAllWindows()

In [30]:
base_model=DenseNet121(weights='imagenet',include_top=False,input_shape=(224,224,3))

In [33]:
cnn = Sequential()
cnn.add(Rescaling(1.0/255.0, input_shape=(224, 224, 3)))  # input_shape 추가
cnn.add(base_model)
cnn.add(Flatten())
cnn.add(Dense(1024, activation='relu'))
cnn.add(Dropout(0.75))
cnn.add(Dense(units=120, activation='softmax'))

cnn.compile(loss='sparse_categorical_crossentropy', 
            optimizer=Adam(learning_rate=0.000001), 
            metrics=['accuracy'])
cnn.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 rescaling_3 (Rescaling)     (None, 224, 224, 3)       0         
                                                                 
 densenet121 (Functional)    (None, 7, 7, 1024)        7037504   
                                                                 
 flatten_3 (Flatten)         (None, 50176)             0         
                                                                 
 dense_6 (Dense)             (None, 1024)              51381248  
                                                                 
 dropout_3 (Dropout)         (None, 1024)              0         
                                                                 
 dense_7 (Dense)             (None, 120)               123000    
                                                                 
Total params: 58,541,752
Trainable params: 58,458,104


In [34]:
hist=cnn.fit(train_ds,epochs=200,validation_data=test_ds,verbose=2)

Epoch 1/200


KeyboardInterrupt: 

In [ ]:
print('정확률=',cnn.evaluate(test_ds,verbose=0)[1]*100)

In [ ]:
!pip install matplotlib
import matplotlib.pyplot as plt

plt.plot(hist.history['accuracy'])
plt.plot(hist.history['val_accuracy'])
plt.title('Accuracy graph')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train','Validation'])
plt.grid()
plt.show()

plt.plot(hist.history['loss'])
plt.plot(hist.history['val_loss'])
plt.title('Loss graph')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train','Validation'])
plt.grid()
plt.show()

In [ ]:
cnn.save('cnn_for_stanford_dogs.h5')	# 미세 조정된 모델을 파일에 저장

import pickle
f=open('dog_species_names.txt','wb')
pickle.dump(train_ds.class_names,f)
f.close()